# User Controller

In [453]:
import uuid
import json
import hashlib
import random
import requests

subscribers_file = 'subscribers.json'

class User(object):
    
    def __init__(self, uuid=None):
        with open(subscribers_file, 'r') as f:
            try:
                self.users = json.load(f)
            except:
                self.users = []
        if uuid:
            return True
                
    def reload(self):
        self.__init__()
    
    def add(self, email):
        self.reload()
        if self.exists(email=email):
            return False
        
        salt = self.salt()
        
        new_user = {
            'uuid': str(uuid.uuid4()),
            'email': email,
            'token': hashlib.sha224((salt + email).encode('UTF-8')).hexdigest(),
            'verified': False,
            'registered': datetime.now().isoformat(),
            'preferences': [1,2,3,4,5],
            'salt': salt,
        }
        
        self.users.append(new_user)
        self.save()
        return new_user
    
    def get(self, uuid=None, email=None, token=None):
        if not uuid and not email and not token:
            return False
        for user in self.users:
            if user['uuid'] == uuid or user['email'] == email or user['token'] == token:
                return user
            
    def get_preferences(self, email):
        return self.get(email=email)['preferences']
            
    def save(self):
        with open(subscribers_file, 'w') as f:
            json.dump(self.users, f)
        self.reload()
        return True
        
    def update_preferences(self, uuid, new_preferences):
        for i, user in enumerate(self.users):
            if user['uuid'] == uuid:
                self.users[i]['preferences'] = new_preferences
                self.save()
                return True
        return False
    
    def generate_token(self, uuid):
        token = self.salt(length=32)
        return "TOKEN"
    
    def email_verification(self, uuid):
        user = self.get(uuid=uuid)
        return requests.post(
            "https://api.mailgun.net/v3/kotek.co/messages",
            auth=("api", "key-0500f40586ec016cbd2c131ea2222ae2"),
            data={"from": "KOTEK.CO <vojtech@kotek.co>",
                "to": user['email'],
                "subject": "Verify your email",
                "template": "email_verification",
                "h:X-Mailgun-Variables": json.dumps(user),
                 })
        
    def update(self, uuid, option, value):
        for i, user in enumerate(self.users):
            if user['uuid'] == uuid:
                self.users[i][option] = value
                self.save()
                self.reload()
                return True
        return False
                
    def verify(self, token):
        for user in self.users:
            if token == user['token']:
                self.update(
                    user['uuid'], 'verified', datetime.now().isoformat()
                )
                return True
        return False
    
    def remove(uuid):
        for i, user in enumerate(self.users):
            if user['uuid'] == uuid:
                return self.users.pop(i)
        return False
            
    def exists(self, email=None, uuid=None):
        """Check whether the given user exists."""
        self.reload()
        for user in self.users:
            if user['email'] == email or user['uuid'] == uuid:
                return True
        return False
    
    def clear(self):
        """Deletes all subscribers from the database."""
        with open(subscribers_file, 'w') as f:
            f.write("")
        self.reload()
        
    def salt(self=None, length=16):
        """Creates random salt of len 16 for user. Not yet implemented."""
        ALPHABET = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
        chars = []
        for char in range(length):
            chars.append(random.choice(ALPHABET))
        return "".join(chars)
  
x = User()
User().users

[{'email': 'kotek.vojtech@gmail.com',
  'preferences': [1, 2, 3, 4, 5],
  'registered': '2019-06-10T00:28:28.903682',
  'salt': '3AKLPFsGgFjzvlDJ',
  'token': 'b11038f3830cab4d68f9181a599a4b4f12f81553619e88d79f0faf8d',
  'uuid': '21466d58-d268-4a42-bf6e-947853fbc1e2',
  'verified': '2019-06-10T00:28:28.909069'},
 {'email': 'vojtech.kotek@merck.com',
  'preferences': [1, 2, 3, 4, 5],
  'registered': '2019-06-10T00:28:28.923401',
  'salt': 'xkrQ7QPmdvDe7yC4',
  'token': '6bfd91fb60d19820a51c80fcce8e41615cf5060da154d03609d882ad',
  'uuid': '4817621b-8127-4d9b-abb7-a234c03c420f',
  'verified': '2019-06-10T00:28:28.924419'},
 {'email': 'drahomira.platlova@merck.com',
  'preferences': [1, 2, 3, 4, 5],
  'registered': '2019-06-10T00:28:28.931836',
  'salt': 'JITuzV9ay7HbBRSO',
  'token': 'e41fa40e579e82fc8cb8e7e187543dea4ce77b4fbd3dae2b5b4e0bae',
  'uuid': 'd3d4883e-5d9c-440c-bcc4-9c44109e54dd',
  'verified': '2019-06-10T00:28:28.938727'},
 {'email': 'vaclav.senych@merck.com',
  'preferences'

In [451]:
x.clear()
x.users

[]

In [452]:
emails = [
    'kotek.vojtech@gmail.com',
    'vojtech.kotek@merck.com',
    'drahomira.platlova@merck.com',
    'vaclav.senych@merck.com',
    'jakub.pospichal@merck.com',
    'lukas.langer1@merck.com',
    'robert.varga@merck.com',
    'lusine.grigoryan@merck.com',
]

for email in emails:
    user = x.add(email)
    x.verify(user['token'])

# x.verify('xxx')
# x.reload()
# x.get(email='kotek.vojtech@gmail.com')
# x.users
# x.update_preferences("this-is-some-weird-uuid", [1,2,3,4,"X"])
# x.email_verification('b1d75ea8-520c-4774-80f5-08457c632ae2')

# User.salt(length=32)

In [415]:
x.verify(z['token'])

26a0895a62aca8f2b86a8003b141a78aa9379873380d29cea57a3605 26a0895a62aca8f2b86a8003b141a78aa9379873380d29cea57a3605
found user!


True

In [390]:
x = {'email': 'kotek.vojtech@gmail.com',
  'preferences': [1, 2, 3, 4, 5],
  'registered': '2019-06-08T23:43:54.789965',
  'token': 'thisisatokenbutnotarealone',
  'uuid': '57688bca-b34f-425c-8fa0-e931fe818901',
  'verified': False}

In [400]:
import hashlib
salt = User.salt()
hash = hashlib.sha224((salt + x['email']).encode('UTF-8')).hexdigest()
print(salt, hash)

17Eq2zEkHgN6YQNK 608f8fad6ccf59f64045b72a474c57a045e3e6af629f3a6cf46eba18


In [125]:
db_structure = {
    'email': 'kotek.vojtech@gmail.com', 
    'token': 'x',
    'verified': True,
    'preferences': [1,2,3,4,5]
}

with open(subscribers_file, 'r') as subs:
    print(subs)

<_io.TextIOWrapper name='subscribers.json' mode='r' encoding='UTF-8'>


In [43]:
preferences = {
    1: "Pastva",
    2: "Sodexo, Riverview",
    3: "Dave B, Weekly",
    4: "Dave B, Daily",
    5: "Potrefena Husa, Na Verandach",
}

users_data = []
# def add_user(email, preferences=None):
#     if email in users_data
    

In [93]:
import uuid as UUID

def user_exists(uuid):
    for user in users_data:
        if user['email'] == email:
            return user['token']
        return False
#     return any([email == user['email'] for user in users_data])

def user_add(email):
    if not user_exists(email):        
        users_data.append({
            'uuid': UUID.uuid4(),
            'email': email, 
            'verified': False,
        })
        return True
    return False
    
def user_remove(uuid):
    if user_exists(uuid):
        for i, user in enumerate(users_data):
            if user['uuid'] == uuid:
                users_data.pop(i)
                return True
    return False

def user_info(uuid):
    if user_exists(uuid):
        for user in users_data:
            if user['uuid'] == uuid:
                return user
    return False

def user_verify(token):
    # 
    if user_exists(email):
        for i, user in enumerate(users_data):
            if user['email'] == email:
                users_data[i]['verified'] == True
                return True
    return False
                
def token_generate(email):
    return "GARGAETGHAETg"
    
email = 'kotek.vojtech@gmail.com'
user_info(email)

False

In [95]:
email = 'kotek.vojtech@gmail.com'
# user_remove(email)
user_add(email)
user_exists(email)

False

In [96]:
users_data

[{'email': 'kotek.vojtech2@gmail.com', 'verified': False},
 {'email': 'kotek.vojtech@gmail.com',
  'uuid': UUID('76657ac9-a384-4c36-9d27-83d1f5549d7a'),
  'verified': False},
 {'email': 'kotek.vojtech@gmail.com',
  'uuid': UUID('e567b9a1-4334-4307-a02f-a5f9a65f7b78'),
  'verified': False}]

In [82]:
import uuid

UUID('17dc8468-8cab-421f-8735-b0acb3872ac3')